## ユースケース: ハイブリッドデプロイメントアーキテクチャ

### シナリオ: インターネットアクセスを伴うセキュアなデータ処理

このデプロイメントパターンでは、AgentCore コンポーネントを分離してセキュリティと機能性のバランスを取ります：

**AgentCore Browser（パブリックモード）**
- インターネットアクセスを持つパブリックサブネットにデプロイ
- 公開されているウェブサイトやサービスのブラウジングを可能に
- 外部統合とリアルタイムデータ取得を管理
- リアルタイム操作のための直接インターネット接続の恩恵

**AgentCore Runtime（VPC モード）**
- セキュアな VPC 内のプライベートサブネットにデプロイ
- 機密データとビジネスロジックを処理
- 厳格なネットワーク分離を維持
- セキュアな内部チャネルを通じて browser コンポーネントと通信

### 利点

- **セキュリティ**: 機密処理がプライベートネットワーク内で分離される
- **パフォーマンス**: Browser 操作が NAT オーバーヘッドなしで直接インターネットアクセスを取得
- **コンプライアンス**: データ分離に関する規制要件を満たす
- **スケーラビリティ**: 各コンポーネントがワークロード需要に基づいて独立してスケール可能

### アーキテクチャフロー

![image.png](Architecture-vpc-browser.png)

# CloudFormation スタック実行手順

## 前提条件
- 適切な権限で設定された AWS CLI
- CloudFormation テンプレートファイルの準備

## 実行手順

### 1. 以下のステップを実行して CFN を起動（約13分）

In [ ]:
import boto3
import yaml
import os

# 設定変数
region = 'us-east-1'  # 希望のリージョンに変更してください
template_path = 'cfn-vpc-browser.yaml'
stack_name = 'vpc-browser-stack-new'

# 設定可能なリージョンで CloudFormation クライアントを初期化
cf_client = boto3.client('cloudformation', region_name=region)

# CloudFormation テンプレートを読み込み
with open(template_path, 'r') as template_file:
    template_body = template_file.read()

try:
    # CloudFormation スタックを作成
    response = cf_client.create_stack(
        StackName=stack_name,
        TemplateBody=template_body,
        Capabilities=['CAPABILITY_IAM', 'CAPABILITY_NAMED_IAM']
    )
    
    print(f"スタック作成を開始しました。リージョン: {region}")
    print(f"スタック ID: {response['StackId']}")
    
    # スタック作成の完了を待機
    waiter = cf_client.get_waiter('stack_create_complete')
    print("スタック作成の完了を待機しています...")
    waiter.wait(StackName=stack_name)
    
    print(f"スタック '{stack_name}' が {region} で正常に作成されました！")
    
except Exception as e:
    print(f"スタック作成エラー: {str(e)}")

### 2. テスト手順

In [ ]:
import boto3
import subprocess
import json
from IPython.display import display, Markdown

# CloudFormation 出力から AgentRuntime ARN を取得
def get_cfn_output(stack_name, output_key, region='us-east-1'):
    """CloudFormation スタックの出力値を取得"""
    cfn = boto3.client('cloudformation', region_name=region)
    try:
        response = cfn.describe_stacks(StackName=stack_name)
        outputs = response['Stacks'][0]['Outputs']
        for output in outputs:
            if output['OutputKey'] == output_key:
                return output['OutputValue']
    except Exception as e:
        print(f"CFN 出力取得エラー: {e}")
    return None

# AgentRuntime ARN を取得（前のセルの stack_name を使用）
agent_runtime_arn = get_cfn_output(stack_name, 'AgentRuntimeArn')
agent_runtime_id = get_cfn_output(stack_name, 'AgentRuntimeId')
development_instance = get_cfn_output(stack_name, 'DevelopmentInstanceId')
web_server_ip = get_cfn_output(stack_name, 'WebServerPrivateIp')


# 完全なテスト手順を生成
instructions = f"""# Bedrock Agent Runtime のテスト手順

## 前提条件
- CloudFormation スタックが正常に完了していること

## ステップバイステップのテストプロセス

### 1. EC2 インスタンスに接続
Browser Connector または SSH で EC2 インスタンス `{development_instance}` に接続

### 2. 環境のセットアップ
EC2 インスタンスで以下のコマンドを実行:

```bash
sudo yum update -y

# 開発ツールをインストール
sudo dnf install git -y && \\
curl -LsSf https://astral.sh/uv/install.sh | sh && \\
echo 'export PATH="$HOME/.cargo/bin:$PATH"' >> ~/.bashrc && \\
source ~/.bashrc && \\
echo 'Install Python Venv'

uv init vpc-browser --python 3.13 && cd vpc-browser
uv venv --python 3.13
source .venv/bin/activate
uv pip install boto3

cat > call-agent.py << 'EOF'
import boto3
import json

client = boto3.client('bedrock-agentcore', region_name='us-east-1')

payload = json.dumps({{
    "prompt": "Access {web_server_ip} over http at port 8080 to check what are holidays in November"
}})

response = client.invoke_agent_runtime(
    agentRuntimeArn="{agent_runtime_arn}",
    runtimeSessionId='dfmeoagmreaklgmrkleafremoigrmtesogmtrskhmtkrlshmt',  # 33文字以上必要
    payload=payload,
    qualifier="DEFAULT"  # オプション
)

response_body = response['response'].read()
response_data = json.loads(response_body)
print("Agent Response:", response_data)
EOF


```

### 3. エージェントテストを実行
```
python call-agent.py
```

### 4. CloudWatch でログを監視:
- AWS コンソールで CloudWatch Logs に移動
- ロググループを検索: /aws/bedrock-agentcore/runtimes/{agent_runtime_id}
- ライブブラウジングを表示するには、コンソール -> Amazon Bedrock AgentCore -> Built-in Tools -> Browser tools -> browser_stack_browser -> View live session
- リアルタイムの実行ログとエラーを監視

"""

Markdown(instructions)

### 3. クリーンアップ

In [ ]:
import boto3

# スタックを削除
cfn = boto3.client('cloudformation', region_name=region)
cfn.delete_stack(StackName=stack_name)

print(f"スタック '{stack_name}' の削除を開始しました。リージョン: '{region}'")

# 削除完了を待機
waiter = cfn.get_waiter('stack_delete_complete')
print("スタック削除の完了を待機しています...")
waiter.wait(StackName=stack_name)

print(f"スタック '{stack_name}' が正常に削除されました！")